##### 1.用PicoDet-l-416模型进行了训练。config地址为work/PaddleDetection_2.3/configs/picodet/picodet_l_416_cocoyml
##### 2.预训练权重为COCO预训练权重
##### 3.数据增强参考PPYOLOV2的数据增强策略
##### 4.学习率调整为0.09，warmup调整为150steps，使用PiecewiseDecay
##### 5.Testing时，对352, 384, 416, 448, 480中的每一尺寸进行测试，挑选出最好的

In [ ]:
# 解压数据集到dataset目录
!mkdir dataset
%cd dataset
!unzip -oq /home/aistudio/data/data105746/train.zip
!unzip -oq /home/aistudio/data/data105747/test.zip
%cd ..

/home/aistudio/dataset
/home/aistudio


In [ ]:
# 对VOC(xml)数据集划分
import os
import shutil
import numpy as np
import pandas as pd
from sklearn import model_selection

data_path = r'/home/aistudio/dataset/train/'

image_path = os.listdir(os.path.join(data_path, 'IMAGES'))
label_path = [i.split('.')[0] + '.xml' for i in image_path]
img_name = ['./IMAGES/' + i for i in image_path]
labels= ['./ANNOTATIONS/' + i for i in label_path]

print(len(img_name),len(labels)) 
ds_train,ds_val, label_train, label_val =model_selection.train_test_split(img_name,labels,test_size= 150, random_state=2021)
ds_train,ds_test, label_train, label_test =model_selection.train_test_split(ds_train,label_train,test_size= 150, random_state=2021)

# 输出train_list.txt和val_list.txt
train_dict = {'img_path': ds_train, 'labels': label_train}
val_dict = {'img_path': ds_val, 'labels': label_val}
test_dict = {'img_path': ds_test, 'labels': label_test}
train_df = pd.DataFrame(train_dict)
val_df = pd.DataFrame(val_dict)
test_df = pd.DataFrame(test_dict)
train_df.to_csv(os.path.join(data_path,'train_list.txt'),sep=' ',header=None,index=None)
val_df.to_csv(os.path.join(data_path,'val_list.txt'),sep=' ',header=None,index=None)
test_df.to_csv(os.path.join(data_path,'test_list.txt'),sep=' ',header=None,index=None)
print(len(train_df),len(val_df),len(test_df))

1400 1400
1100 150 150


In [ ]:
# 模型训练
%cd /home/aistudio/work/PaddleDetection_2.3
!python tools/train.py -c configs/picodet/picodet_l_416_coco.yml --eval -o pretrain_weights=output/picodet_l_416_coco/99


/home/aistudio/work/PaddleDetection_2.3
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `

In [ ]:
# 模型推理
!python tools/infer.py -c configs/picodet/picodet_l_416_coco.yml --infer_dir=/home/aistudio/dataset/test/IMAGES/ --output_dir=infer_output/ --draw_threshold=0  --save_txt=True \
-o weights=/home/aistudio/work/PaddleDetection_2.3/output/picodet_l_416_coco/best_model --draw_threshold=0

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the

In [ ]:
import csv
import os
headers = ['image_id','bbox','category_id','confidence']
classList = ['crazing','inclusion','pitted_surface','scratches','patches','rolled-in_scale']
rows = []

rootdir = '/home/aistudio/work/PaddleDetection/infer_output'
list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
for i in range(0,len(list)):
       path = os.path.join(rootdir,list[i])
       if os.path.isfile(path) and path.endswith('txt'):
           txtFile = open(path)
           print(path)
           result = txtFile.readlines()
           for r in result:
               ls = r.split(' ')
               Cls = ls[0]
               sco = float(ls[1])
               xmin = float(ls[2])
               ymin = float(ls[3])
               w = float(ls[4])
               h = float(ls[5])
               xmax = xmin+w
               ymax = ymin+h
               clsID = classList.index(Cls)
               imgID = list[i][:-4]
               row = [imgID,[xmin,ymin,xmax,ymax],clsID,sco]
               rows.append(row)
with open('submission.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

/home/aistudio/PaddleDetection/infer_output/1668.txt
/home/aistudio/PaddleDetection/infer_output/1792.txt
/home/aistudio/PaddleDetection/infer_output/1643.txt
/home/aistudio/PaddleDetection/infer_output/1501.txt
/home/aistudio/PaddleDetection/infer_output/1431.txt
/home/aistudio/PaddleDetection/infer_output/1624.txt
/home/aistudio/PaddleDetection/infer_output/1429.txt
/home/aistudio/PaddleDetection/infer_output/1666.txt
/home/aistudio/PaddleDetection/infer_output/1760.txt
/home/aistudio/PaddleDetection/infer_output/1707.txt
/home/aistudio/PaddleDetection/infer_output/1514.txt
/home/aistudio/PaddleDetection/infer_output/1703.txt
/home/aistudio/PaddleDetection/infer_output/1614.txt
/home/aistudio/PaddleDetection/infer_output/1710.txt
/home/aistudio/PaddleDetection/infer_output/1449.txt
/home/aistudio/PaddleDetection/infer_output/1601.txt
/home/aistudio/PaddleDetection/infer_output/1793.txt
/home/aistudio/PaddleDetection/infer_output/1770.txt
/home/aistudio/PaddleDetection/infer_output/16